# Data Preprocessing
### Dataset: [NFL Big Data Bowl 2019](https://github.com/nfl-football-ops/Big-Data-Bowl/tree/master/Data)

In [40]:
import pandas as pd
import numpy as np
import torch
import tqdm 
from IPython.display import display, Markdown
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import compose,  model_selection
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import joblib

In [6]:
games = pd.read_csv("data/games.csv")
players = pd.read_csv("data/players.csv")
plays = pd.read_csv("data/plays.csv")

In [7]:
plays.head()

,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,...,VisitorScoreAfterPlay,isPenalty,isSTPlay,SpecialTeamsPlayType,KickReturnYardage,PassLength,PassResult,YardsAfterCatch,PlayResult,playDescription
0,2017091004,37,1,15:00:00,0,0,DET,DET,35.0,NaN,...,0,False,True,Kickoff,23.0,NaN,NaN,NaN,42,K.Redfern kicks 65 yards from DET 35 to ARZ 0....
1,2017091004,73,1,14:54:00,1,10,ARI,ARI,23.0,SHOTGUN,...,0,False,False,NaN,NaN,-2.0,C,6.0,4,(14:54) (Shotgun) C.Palmer pass short right to...
2,2017091004,97,1,14:16:00,2,6,ARI,ARI,27.0,SINGLEBACK,...,0,False,False,NaN,NaN,NaN,NaN,NaN,2,(14:16) Da.Johnson up the middle to ARZ 29 for...
3,2017091004,118,1,13:34:00,3,4,ARI,ARI,29.0,SHOTGUN,...,0,False,False,NaN,NaN,7.0,C,7.0,15,(13:34) (Shotgun) C.Palmer pass short middle t...
4,2017091004,153,1,12:51:00,1,10,ARI,ARI,44.0,SHOTGUN,...,0,False,False,NaN,NaN,NaN,NaN,NaN,-2,(12:51) (Shotgun) Da.Johnson left end to ARZ 4...


In [8]:
with open("data/schema.md", "r") as file:
    markdown_content = file.read()

display(Markdown(markdown_content))


<!-- README.md is generated from README.Rmd. Please edit that file -->
Summary of data
---------------

Here, you'll find a summary of each data set in the 2019 Data Bowl, a list of *key* variables to join on, and a description of each variable.

File descriptions
-----------------

Game data: The `games.csv` file contains game-level information for each game from the first 6 weeks of the 2017 season. The *key* variable is **`gameId`**.

Play data: The `plays.csv` file contains play-level information from each game from the first 6 weeks of the 2017 season. The *key* variables are **`gameId`** and **`playId`**.

Player data: The `players.csv` file contains player-level information from players that participated in at least one play during the first six weeks of the 2017 regular season. The *key* variable is **`nflId`**.

Tracking data: Files `tracking_gameId_[gameId].csv` contain player tracking data from game `[gameId]`. Nearly all plays from `[gameId]` are included; certain plays with incomplete or missing data are dropped. The *key* variables are **`gameId`**, **`playId`**, and **`nflId`**.

Game data
---------

-   `season`: Season of game (numeric)
-   `week`: Week of game, 1 through 6 (numeric)
-   `gameDate`: Game Date (time, mm/dd/yyyy)
-   **`gameId`**: Game identifier, unique (numeric)
-   `gameTimeEastern`: Start time of game (time, HH:MM:SS, EST)
-   `HomeScore`: Final score for the home team (numeric)
-   `VisitorScore`: Final score for the away team (numeric)
-   `homeTeamAbbr`: Home team three-letter code (text)
-   `visitorTeamAbbr`: Visiting team three-letter code (text)
-   `homeDisplayName`: Home team name (text)
-   `visitorDisplayName`: Visiting team name (text)
-   `Stadium`: Stadium (text)
-   `Location`: City (text)
-   `StadiumType`: Type of stadium (text)
-   `Turf`: Surface of stadium (text)
-   `GameLength`: Time the game took to complete (time, HH:MM:SS)
-   `GameWeather`: Game weather (text)
-   `Temperature`: Temperature in Fahrenheit, drawn roughly at the start of the game (numeric)
-   `Humidity`: Humidity (numeric)
-   `WindSpeed`: Wind speed, in miles-per-hour (numeric)
-   `WindDirection`: Direction of wind (text)

Play data
---------

-   **`gameId`**: Game identifier, unique (numeric)
-   **`playId`**: Play identifier, not unique across games (numeric)
-   `quarter`: Game quarter (numeric)
-   `GameClock`: Time on game clock at start of play (time, counting down from 15:00, MM:SS)
-   `down`: Down (numeric)
-   `yardsToGo`: Distance needed for a first down (numeric)
-   `yardlineSide`: 3-letter team code corresponding to line-of-scrimmage (text)
-   `yardlineNumber`: Yard line at line-of-scrimmage (numeric)
-   `personnel.offense`: Personnel used by offensive team (text)
-   `defendersInTheBox`: Number of defenders in close proximity to line-of-scrimmage (numeric)
-   `numberOfPassRushers`: Number of pass rushers (numeric)
-   `personnel.defense`: Personnel used by defensive team (text)
-   `HomeScoreBeforePlay`: Home score prior to the play (numeric)
-   `VisitorScoreBeforePlay`: Visiting team points at the end of the play (numeric)
-   `HomeScoreAfterPlay`: Home team points at the end of the play (numeric)
-   `VisitorScoreAfterPlay`: Visitor team points at the end of the play (numeric)
-   `isPenalty`: TRUE/FALSE for whether or not a penalty was called on the play (binary)
-   `isSTPlay`: TRUE/FALSE for whether or not the play is labelled a special teams play (binary)
-   `SpecialTeamsPlayType`: Type of play if `isSTPlay == TRUE` (text)
-   `KickReturnYardage`: Return yardage among special teams plays (numeric)
-   `PassLength`: Pass length, in yards (numeric)
-   `PassResult`: Result of pass play (text, `C`: caught, `I`: incomplete, `IN`: intercepted, `R`: run, `S`: sack)
-   `YardsAfterCatch`: Yardage receiver gained after a pass completion (numeric)
-   `PlayResult`: Result of play, in yards (numeric)
-   `playDescription`: Description of play (text)

Player data
-----------

-   **`nflId`**: Player identification number, unique across players (numeric)
-   `FirstName`: First name of player (text)
-   `LastName`: Last name of player (text)
-   `PositionAbbr`: Position of player (text)
-   `EntryYear`: Year in which player entered NFL (numeric)
-   `DraftRound`: Round in which player was drafted --`NULL` for players not drafted (numeric)
-   `DraftNumber`: Overall pick number among drafted players (numeric)
-   `Height`: Player height in feet/inches (text)
-   `Weight`: Player weight in pounds (numeric)
-   `College`: Player college (text)

Tracking data
-------------

Files `tracking_gameId_[gameId].csv` contains player tracking data from game `[gameId]`. Nearly all plays from `[gameId]` are included; certain plays with insufficient data are dropped.

-   `time`: Time stamp of play (time, yyyy-mm-dd, hh:mm:ss)
-   `x`: Player position along the long axis of the field, 0 - 120 yards. See Figure 1 below. (numeric)
-   `y`: Player position along the short axis of the field, 0 - 53.3 yards. See Figure 1 below. (numeric)
-   `s`: Speed in yards/second (numeric)
-   `dis`: Distance traveled from prior time point, in yards (numeric)
-   `dir`: Angle of player motion (deg), 0 - 360 degrees (numeric)
-   `event`: Tagged play details, including moment of ball snap, pass release, pass catch, tackle, etc (text)
-   **`nflId`**: Player identification number, unique across players (numeric)
-   `displayName`: Player name (text)
-   `jerseyNumber`: Jersey number of player (numeric)
-   `team`: Team (away or home) of corresponding player (text)
-   `frame.id`: Frame identifier for each play, starting at 1 (numeric)
-   **`gameId`**: Game identifier, unique (numeric)
-   **`playId`**: Play identifier, not unique across games (numeric)

<img src="Extras/Fig1.PNG" align="right" />


In [9]:
df = plays.merge(games, on="gameId", how="left")

df.columns

Index(['gameId', 'playId', 'quarter', 'GameClock', 'down', 'yardsToGo',
       'possessionTeam', 'yardlineSide', 'yardlineNumber', 'offenseFormation',
       'personnel.offense', 'defendersInTheBox', 'numberOfPassRushers',
       'personnel.defense', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay',
       'HomeScoreAfterPlay', 'VisitorScoreAfterPlay', 'isPenalty', 'isSTPlay',
       'SpecialTeamsPlayType', 'KickReturnYardage', 'PassLength', 'PassResult',
       'YardsAfterCatch', 'PlayResult', 'playDescription', 'season', 'week',
       'gameDate', 'gameTimeEastern', 'HomeScore', 'VisitorScore',
       'homeTeamAbbr', 'visitorTeamAbbr', 'homeDisplayName',
       'visitorDisplayName', 'Stadium', 'Location', 'StadiumType', 'Turf',
       'GameLength', 'GameWeather', 'Temperature', 'Humidity', 'WindSpeed',
       'WindDirection'],
      dtype='object')

In [10]:
# Filter out special teams plays (focus on offensive plays)
opdf = df[df["isSTPlay"] == False].copy()

print(opdf.shape)
opdf.head()


(11558, 47)


,gameId,playId,quarter,GameClock,down,yardsToGo,possessionTeam,yardlineSide,yardlineNumber,offenseFormation,...,Stadium,Location,StadiumType,Turf,GameLength,GameWeather,Temperature,Humidity,WindSpeed,WindDirection
1,2017091004,73,1,14:54:00,1,10,ARI,ARI,23.0,SHOTGUN,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN
2,2017091004,97,1,14:16:00,2,6,ARI,ARI,27.0,SINGLEBACK,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN
3,2017091004,118,1,13:34:00,3,4,ARI,ARI,29.0,SHOTGUN,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN
4,2017091004,153,1,12:51:00,1,10,ARI,ARI,44.0,SHOTGUN,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN
5,2017091004,174,1,12:13:00,2,12,ARI,ARI,42.0,EMPTY,...,Ford Field,"Detroit, MI",Indoors,Field Turf,03:26:00,Controlled Climate,68.0,70.0,NaN,NaN


In [11]:
# Convert GameClock (MM:SS) to seconds
def clock_to_seconds(clock):
    if pd.isna(clock):
        return np.nan
    minutes, seconds, ms = map(int, clock.split(":"))
    return minutes * 60 + seconds


opdf["GameClockSeconds"] = opdf["GameClock"].apply(clock_to_seconds)
opdf["GameClockSeconds"][:1] #clock is now in seconds

1    894
Name: GameClockSeconds, dtype: int64

In [12]:
# Drop rows where PlayResult is missing (target variable)
opdf = opdf.dropna(subset=["PlayResult"])

# Drop post-play information to avoid data leakage

opdf = opdf.drop(labels=[
    'HomeScoreAfterPlay', 'VisitorScoreAfterPlay', 
    'PassLength', 'YardsAfterCatch', 'GameClock', 'PassResult',
    'playDescription', 'gameDate', 'gameTimeEastern', 'homeTeamAbbr',
    'visitorTeamAbbr', 'homeDisplayName', 'visitorDisplayName', 'Stadium',
    'Location', 'GameLength', 'season', 'week', 'HomeScore', 'VisitorScore',
    'isSTPlay', 'SpecialTeamsPlayType', 'WindSpeed', 'WindDirection','KickReturnYardage', 
    'possessionTeam','yardlineSide'], axis=1)

In [13]:
# Select numerical columns
numcols = opdf.select_dtypes(include=np.number).columns.tolist()

# Select categorical columns (anything that's NOT numeric)
catcols = opdf.select_dtypes(exclude=np.number).columns.tolist()

print("Numerical Columns:", numcols)
print("Categorical Columns:", catcols)

Numerical Columns: ['gameId', 'playId', 'quarter', 'down', 'yardsToGo', 'yardlineNumber', 'defendersInTheBox', 'numberOfPassRushers', 'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'PlayResult', 'Temperature', 'Humidity', 'GameClockSeconds']
Categorical Columns: ['offenseFormation', 'personnel.offense', 'personnel.defense', 'isPenalty', 'StadiumType', 'Turf', 'GameWeather']


In [14]:
#Fill missing numeric values with median

opdf[numcols] = opdf[numcols].fillna(opdf[numcols].median())

In [15]:
# Fill missing categorical features with a placeholder

opdf[catcols] = opdf[catcols].fillna("Unknown")

In [16]:
opdf.head()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,...,HomeScoreBeforePlay,VisitorScoreBeforePlay,isPenalty,PlayResult,StadiumType,Turf,GameWeather,Temperature,Humidity,GameClockSeconds
1,2017091004,73,1,1,10,23.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,...,0,0,False,4,Indoors,Field Turf,Controlled Climate,68.0,70.0,894
2,2017091004,97,1,2,6,27.0,SINGLEBACK,"1 RB, 1 TE, 3 WR",6.0,4.0,...,0,0,False,2,Indoors,Field Turf,Controlled Climate,68.0,70.0,856
3,2017091004,118,1,3,4,29.0,SHOTGUN,"1 RB, 0 TE, 4 WR",5.0,4.0,...,0,0,False,15,Indoors,Field Turf,Controlled Climate,68.0,70.0,814
4,2017091004,153,1,1,10,44.0,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,4.0,...,0,0,False,-2,Indoors,Field Turf,Controlled Climate,68.0,70.0,771
5,2017091004,174,1,2,12,42.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,4.0,...,0,0,True,0,Indoors,Field Turf,Controlled Climate,68.0,70.0,733


In [17]:
def extract_personnel_counts(personnel_str, position):
    import re
    match = re.search(rf"(\d+) {position}", personnel_str)
    return int(match.group(1)) if match else 0

# Create offensive personnel columns
opdf["num_RB"] = opdf["personnel.offense"].apply(lambda x: extract_personnel_counts(x, "RB"))
opdf["num_TE"] = opdf["personnel.offense"].apply(lambda x: extract_personnel_counts(x, "TE"))
opdf["num_WR"] = opdf["personnel.offense"].apply(lambda x: extract_personnel_counts(x, "WR"))

# Create defensive personnel columns
opdf["num_DL"] = opdf["personnel.defense"].apply(lambda x: extract_personnel_counts(x, "DL"))
opdf["num_LB"] = opdf["personnel.defense"].apply(lambda x: extract_personnel_counts(x, "LB"))
opdf["num_DB"] = opdf["personnel.defense"].apply(lambda x: extract_personnel_counts(x, "DB"))

opdf.head()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,offenseFormation,personnel.offense,defendersInTheBox,numberOfPassRushers,...,GameWeather,Temperature,Humidity,GameClockSeconds,num_RB,num_TE,num_WR,num_DL,num_LB,num_DB
1,2017091004,73,1,1,10,23.0,SHOTGUN,"1 RB, 1 TE, 3 WR",6.0,4.0,...,Controlled Climate,68.0,70.0,894,1,1,3,4,2,5
2,2017091004,97,1,2,6,27.0,SINGLEBACK,"1 RB, 1 TE, 3 WR",6.0,4.0,...,Controlled Climate,68.0,70.0,856,1,1,3,4,2,5
3,2017091004,118,1,3,4,29.0,SHOTGUN,"1 RB, 0 TE, 4 WR",5.0,4.0,...,Controlled Climate,68.0,70.0,814,1,0,4,4,1,6
4,2017091004,153,1,1,10,44.0,SHOTGUN,"1 RB, 2 TE, 2 WR",7.0,4.0,...,Controlled Climate,68.0,70.0,771,1,2,2,4,3,4
5,2017091004,174,1,2,12,42.0,EMPTY,"1 RB, 2 TE, 2 WR",6.0,4.0,...,Controlled Climate,68.0,70.0,733,1,2,2,4,3,4


In [18]:
opdf = opdf.drop(labels = ['personnel.offense', 'personnel.defense'], axis = 1)

opdf.head()

,gameId,playId,quarter,down,yardsToGo,yardlineNumber,offenseFormation,defendersInTheBox,numberOfPassRushers,HomeScoreBeforePlay,...,GameWeather,Temperature,Humidity,GameClockSeconds,num_RB,num_TE,num_WR,num_DL,num_LB,num_DB
1,2017091004,73,1,1,10,23.0,SHOTGUN,6.0,4.0,0,...,Controlled Climate,68.0,70.0,894,1,1,3,4,2,5
2,2017091004,97,1,2,6,27.0,SINGLEBACK,6.0,4.0,0,...,Controlled Climate,68.0,70.0,856,1,1,3,4,2,5
3,2017091004,118,1,3,4,29.0,SHOTGUN,5.0,4.0,0,...,Controlled Climate,68.0,70.0,814,1,0,4,4,1,6
4,2017091004,153,1,1,10,44.0,SHOTGUN,7.0,4.0,0,...,Controlled Climate,68.0,70.0,771,1,2,2,4,3,4
5,2017091004,174,1,2,12,42.0,EMPTY,6.0,4.0,0,...,Controlled Climate,68.0,70.0,733,1,2,2,4,3,4


In [19]:
offenseFormation = pd.DataFrame(opdf['offenseFormation'])
offenseFormation

,offenseFormation
1,SHOTGUN
2,SINGLEBACK
3,SHOTGUN
4,SHOTGUN
5,EMPTY
...,...
14187,SINGLEBACK
14188,SHOTGUN
14189,I_FORM
14190,I_FORM


## Data Preprocessing for All Models


In [ ]:
X = opdf.drop('PlayResult', axis=1)
y = opdf['PlayResult']


categorical_features= X.select_dtypes(exclude=np.number).columns.tolist()
numerical_features = X.select_dtypes(include=np.number).columns.tolist()


lrmTransformer = compose.ColumnTransformer(
    transformers=[
        ('numerical', StandardScaler(), numerical_features),
        ('categorical', OneHotEncoder(sparse_output=True), categorical_features)
    ]
)

# Split data

"""
Train: 80% of total data
Validation: 10% of total data (50% of 20%)
Test: 10% of total data (50% of 20%)
"""

# Single dataset for other models (seed 102)
print("\nGenerating single dataset for other models (seed 102)...")
X_train, X_temp, y_train, y_temp = model_selection.train_test_split(
    X, y, train_size=0.8, random_state=102
)
X_valid, X_test, y_valid, y_test = model_selection.train_test_split(
    X_temp, y_temp, train_size=0.5, random_state=102
)

# Transform single dataset
X_train_transformed = lrmTransformer.fit_transform(X_train)
X_valid_transformed = lrmTransformer.transform(X_valid)
X_test_transformed = lrmTransformer.transform(X_test)


# Save single dataset as .pkl files for other models
joblib.dump(X_train_transformed, 'X_train_transformed.pkl')
joblib.dump(X_valid_transformed, 'X_valid_transformed.pkl')
joblib.dump(X_test_transformed, 'X_test_transformed.pkl')
joblib.dump(y_train, 'y_train.pkl')
joblib.dump(y_valid, 'y_valid.pkl')
joblib.dump(y_test, 'y_test.pkl')

print(f"Single dataset - Train shape: {X_train_transformed.shape}, "
      f"Valid shape: {X_valid_transformed.shape}, Test shape: {X_test_transformed.shape}")

# Six datasets for neural network with different seeds
random_seeds = [102, 203, 305, 405, 506, 607]

for seed in random_seeds:
    print(f"\nGenerating tensor dataset for neural net with seed: {seed}")
    #split
    X_train_nn, X_temp_nn, y_train_nn, y_temp_nn = model_selection.train_test_split(
        X, y, train_size=0.8, random_state=seed
    )
    X_valid_nn, X_test_nn, y_valid_nn, y_test_nn = model_selection.train_test_split(
        X_temp_nn, y_temp_nn, train_size=0.5, random_state=seed
    )

    # Transform
    X_train_transformed_nn = lrmTransformer.fit_transform(X_train_nn)
    X_valid_transformed_nn = lrmTransformer.transform(X_valid_nn)
    X_test_transformed_nn = lrmTransformer.transform(X_test_nn)


    # Convert sparse to dense
    if hasattr(X_train_transformed_nn, 'toarray'):
        X_train_transformed_nn = X_train_transformed_nn.toarray()
        X_valid_transformed_nn = X_valid_transformed_nn.toarray()
        X_test_transformed_nn = X_test_transformed_nn.toarray()

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train_transformed_nn)
    y_train_tensor = torch.FloatTensor(y_train_nn.values).view(-1, 1)
    X_valid_tensor = torch.FloatTensor(X_valid_transformed_nn)
    y_valid_tensor = torch.FloatTensor(y_valid_nn.values).view(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_transformed_nn)
    y_test_tensor = torch.FloatTensor(y_test_nn.values).view(-1, 1)

    # Save tensors as .pt files
    torch.save(X_train_tensor, f'X_train_seed{seed}.pt')
    torch.save(y_train_tensor, f'y_train_seed{seed}.pt')
    torch.save(X_valid_tensor, f'X_valid_seed{seed}.pt')
    torch.save(y_valid_tensor, f'y_valid_seed{seed}.pt')
    torch.save(X_test_tensor, f'X_test_seed{seed}.pt')
    torch.save(y_test_tensor, f'y_test_seed{seed}.pt')

    print(f"Seed {seed} - Train shape: {X_train_transformed_nn.shape}, "
          f"Valid shape: {X_valid_transformed_nn.shape}, Test shape: {X_test_transformed_nn.shape}")

joblib.dump(lrmTransformer, 'transformer.pkl')

print("\nPreprocessing complete. Saved single dataset (.pkl) for other models and six tensor datasets (.pt) for neural net.")



Generating single dataset for other models (seed 102)...
Single dataset - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 102
Seed 102 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 203
Seed 203 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 305
Seed 305 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 405
Seed 405 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 506
Seed 506 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Generating tensor dataset for neural net with seed: 607
Seed 607 - Train shape: (9246, 77), Valid shape: (1156, 77), Test shape: (1156, 77)

Prepr